In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.1'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [770 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-05-18 05:43:32--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.12MB/s    in 0.8s    

2022-05-18 05:43:34 (1.12 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Amazon_Automotive").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "<RDS Endpoint>"
spark.sparkContext.addFile(url)
df = spark.read.option("sep","\t").option("header","true").option("inferSchema","true").csv(SparkFiles.get("amazon_reviews_us_Automotive_v1_00.tsv"))

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

In [5]:
# Examine schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [6]:
# Get row count of dataframe
df.count()

3514942

In [7]:
# Drop null values
dropna_df = df.dropna()
dropna_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

In [8]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

In [9]:
# Create user dataframe to match active_user table
customers_df = dropna_df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)","customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   28258386|             3|
|   18634862|             3|
|    2072312|             3|
|   33132797|            15|
|   14539137|             1|
|   14552054|             5|
|    9347763|             1|
|   43626894|             2|
|   27314089|            47|
|   36829268|             1|
|   24540309|             2|
|    8501205|             1|
|    4361670|             2|
|   30416085|             2|
|    8306731|             1|
|   11491871|             1|
|   46406103|             1|
|   18417791|             1|
|   51299913|             1|
|   20587872|             4|
+-----------+--------------+
only showing top 20 rows



In [10]:
products_df = dropna_df.select(["product_id","product_title"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00LPRXQ4Y|17" 2003-2006 For...|
|B000C7S0TO|Spectra Premium C...|
|B000CO9WE4|K&N E-4665 High P...|
|B000GKD5NI|Suncutters Rear W...|
|B009SDA7TE|Lug Nuts Landcrui...|
|B00KV15KRG|Fits 2007-2009 Ho...|
|B0002JMAKW|Castrol 12614 Dot...|
|B00XJKMM6S|New Power Window ...|
|B000C5CEKC|Motorad MGC-791 L...|
|B00GD9R2A8|Genuine Ford AL3Z...|
|B00RM327NG|Air Vent Smartpho...|
|B0071NL7UQ|FDJ New Replaceme...|
|B00DSOL0FY|Omix-Ada 17402.09...|
|B00GRV48TK|GMB 530-2480 Fuel...|
|B000E4PCGE|00-03 NISSAN SENT...|
|B005Z8HAUK|Slime 40026 2X He...|
|B0043AV4NK|MegaBrand N1 4" B...|
|B00CDHD76C|Jackly 45-in-One ...|
|B000CF1XO0|Bosch 3397118942 ...|
|B003VLBPJA|Heavy-Duty 15" Ca...|
+----------+--------------------+
only showing top 20 rows



In [11]:
reviews_df = dropna_df.select(["review_id","customer_id","product_id","product_parent","review_date"])
reviews_df.show()


+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RAB23OVFNCXZQ|   36075342|B00LPRXQ4Y|     339193102| 2015-08-31|
|R3NORADVJO6IE6|   42462164|B000C7S0TO|     907684644| 2015-08-31|
|R299F4SO98S5OO|   21241933|B000CO9WE4|     752246352| 2015-08-31|
|R2DA9DOT03UW6I|   52570308|B000GKD5NI|     105401756| 2015-08-31|
|R2OGCH681EQHU6|   38200102|B009SDA7TE|     728471129| 2015-08-31|
| R2JMKIC16MHD7|   34866169|B00KV15KRG|     962286893| 2015-08-31|
|R1DB5DA7CWWTI8|     184627|B0002JMAKW|     267002949| 2015-08-31|
|R1N8XWFDK4QACP|    2975964|B00XJKMM6S|     570789093| 2015-08-31|
|R19VFLX6MRU0CX|   40946484|B000C5CEKC|     389524802| 2015-08-31|
| RLTD8YDGTP9PT|   12554469|B00GD9R2A8|      21425394| 2015-08-31|
| R78B5LCFJ52SR|   35335277|B00RM327NG|     816815445| 2015-08-31|
|R33SKWL0HEQIQ9|   44957003|B0071NL7UQ|     192225755| 2015-08

In [12]:
from pyspark.sql.functions import to_date

In [13]:
#Review Date needs to be changed here
reviews_df = reviews_df.select(["review_id","customer_id","product_id","product_parent",to_date("review_date",'yyyy-MM-dd').alias("review_date")])
reviews_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RAB23OVFNCXZQ|   36075342|B00LPRXQ4Y|     339193102| 2015-08-31|
|R3NORADVJO6IE6|   42462164|B000C7S0TO|     907684644| 2015-08-31|
|R299F4SO98S5OO|   21241933|B000CO9WE4|     752246352| 2015-08-31|
|R2DA9DOT03UW6I|   52570308|B000GKD5NI|     105401756| 2015-08-31|
|R2OGCH681EQHU6|   38200102|B009SDA7TE|     728471129| 2015-08-31|
| R2JMKIC16MHD7|   34866169|B00KV15KRG|     962286893| 2015-08-31|
|R1DB5DA7CWWTI8|     184627|B0002JMAKW|     267002949| 2015-08-31|
|R1N8XWFDK4QACP|    2975964|B00XJKMM6S|     570789093| 2015-08-31|
|R19VFLX6MRU0CX|   40946484|B000C5CEKC|     389524802| 2015-08-31|
| RLTD8YDGTP9PT|   12554469|B00GD9R2A8|      21425394| 2015-08-31|
| R78B5LCFJ52SR|   35335277|B00RM327NG|     816815445| 2015-08-31|
|R33SKWL0HEQIQ9|   44957003|B0071NL7UQ|     192225755| 2015-08

In [14]:
vine_df = dropna_df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RAB23OVFNCXZQ|          1|            0|          0|   N|
|R3NORADVJO6IE6|          5|            0|          0|   N|
|R299F4SO98S5OO|          5|            1|          1|   N|
|R2DA9DOT03UW6I|          5|            2|          3|   N|
|R2OGCH681EQHU6|          5|            0|          0|   N|
| R2JMKIC16MHD7|          5|            2|          2|   N|
|R1DB5DA7CWWTI8|          5|            0|          0|   N|
|R1N8XWFDK4QACP|          5|            0|          0|   N|
|R19VFLX6MRU0CX|          5|            0|          0|   N|
| RLTD8YDGTP9PT|          1|            0|          0|   N|
| R78B5LCFJ52SR|          4|            0|          0|   N|
|R33SKWL0HEQIQ9|          5|            0|          0|   N|
|R1HIOV0UX7P13D|          5|            0|          0|   N|
| R4WS0E0MSP9DH|          5|            

Postgres Setup

In [15]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<s3 bucket>/<server name>"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [16]:
# Complete
# Write DataFrame to customers table in RDS

customers_df.write.jdbc(url=jdbc_url, table='customers', mode='overwrite', properties=config)

In [17]:
# Complete
# Write DataFrame to products table in RDS

products_df.write.jdbc(url=jdbc_url, table='products', mode='overwrite', properties=config)

In [ ]:
# Completed
# Write DataFrame to review_id_table table in RDS

reviews_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Completed
# Write DataFrame to vine_table table in RDS

vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)